In [2]:
import dagshub
import mlflow
import os
import time
import csv
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import h2o
import glob
import gc
import matplotlib.pyplot as plt
from multiprocessing import Pool
from mlflow.models.signature import infer_signature
import csv
import json
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,accuracy_score,log_loss

In [3]:
# from model_trainer_script import ModelTrainer

In [4]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'tnbmarketplace'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd7c1a451bfc46bf96fbbe5d723becf3955dfa2b3'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/tnbmarketplace/instacart_basket_analysis_exp_tracking.mlflow'
!dagshub login --token "d7c1a451bfc46bf96fbbe5d723becf3955dfa2b3"

2024-11-11 12:58:05,989 - [INFO]  - [_client.py:1026] - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"
✅ Token added successfully


In [5]:
dagshub.init(repo_name = "instacart_basket_analysis_exp_tracking",repo_owner = 'tnbmarketplace')

Accessing as tnbmarketplace

Initialized MLflow to track repo "tnbmarketplace/instacart_basket_analysis_exp_tracking"

Repository tnbmarketplace/instacart_basket_analysis_exp_tracking initialized!

In [6]:
# !ngrok authtoken "2kvaYw5ZiG5bL8iM8YJBVJPk1Ru_3C16mMgmpKEBYb28PPLUe"
# # Start Ngrok tunnel on H2O's port
# ngrok_tunnel = ngrok.connect(8787)
# print("H2O UI accessible at:", ngrok_tunnel)

In [7]:
def get_time(start):
    return time.time() - start

In [8]:
dataset_version = "1.2" # dataset . split method
model_version = "1.1.1" # algorithm . param version . used dataset 
read_as_xgb_dmatrix =  True
train_xgb_gbm= True
train_xgb_rf = False

In [9]:

train_features_name = []
with open("/kaggle/input/final-dataset-generator/train_set_columns.txt","r") as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        train_features_name = row
train_features_name.remove('') 

test_features_name = []
with open("/kaggle/input/final-dataset-generator/test_set_columns.txt","r") as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        test_features_name = row
test_features_name.remove('')  

train_label_index = train_features_name.index('reordered')
# test_label_index = test_features_name.index('reordered')



for i in range(len(test_features_name)):

    if test_features_name[i] == 'time_mean_dow_count':
       test_features_name[i] ='total_ord_count_p_dow'

    if test_features_name[i] == 'time_mean_ohod_count':
       test_features_name[i] ='total_ord_count_p_ohod'


In [10]:
xgb_booster = mlflow.xgboost.load_model("mlflow-artifacts:/c872cb0ab0ca48d481a71a9bcd2a2102/626567462ef746f8b45a4255ff51c5d1/artifacts/xgb_gbm_model")


In [11]:
params = {
    'max_depth': 1,
    'min_child_weight': 389,
    'verbose': -1,
    'gamma': 438,
    'eta': 0.0907183045377987,
    'subsample': 0.10741019033611729,
    'colsample_bytree': 0.336917979846298,
    'colsample_bylevel': 0.118177830385349,
    'lambda': 11,
    'alpha': 89,
    'booster': 'gbtree',
    'tree_method': 'hist',
    'objective': 'binary:logistic'
}


In [12]:
if read_as_xgb_dmatrix == True:
    
    train_features_name.remove('reordered')
    # test_features_name.remove('reordered')

    dtrain_2 = xgb.DMatrix(f"/kaggle/input/final-dataset-generator/final_prior_train_set.csv/part-00000-37c713df-e906-4480-9322-5a9f58652182-c000.csv?format=csv&label_column={train_label_index}",nthread=-1,feature_names=train_features_name)
    # dtest_2 = xgb.DMatrix(f"/kaggle/input/final-dataset-generator/featured_test_set.csv/part-00000-16b6977e-6b54-4aff-ad77-cb8ae65dfb2f-c000.csv?format=csv&label_column={test_label_index}",nthread=-1,feature_names=test_features_name)


In [13]:
# %% [code]
import time
import os
import gc
import json
import mlflow
import h2o
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, log_loss
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OGradientBoostingEstimator

class ModelTrainer:
    
    def __init__(self, experiment_name, train_set, test_set=None, target_column='reordered'):
        """
        Initializes the ModelTrainer with the experiment name, training set, and optional test set.
        
        Parameters:
        experiment_name (str): The name of the MLflow experiment.
        train_set (H2OFrame): The training dataset.
        test_set (H2OFrame, optional): The test dataset. Defaults to None.
        target_column (str): The name of the target column. Defaults to 'reordered'.
        """
        self.train_set = train_set
        self.test_set = test_set
        self.exp_name = experiment_name
        self.target_column = target_column  # Store the target column name
        mlflow.set_experiment(self.exp_name)

    def __log_details(self, y_true, preds, prev_commit_hash, params, model=None):
        """
        Logs detailed information about the model's performance, environment, and dataset to MLflow.
        
        Parameters:
        y_true (array-like): True values for the target variable.
        preds (array-like): Predicted values.
        prev_commit_hash (str): The commit hash for version control reference.
        params (dict): Model hyperparameters.
        model (object, optional): The trained model. Defaults to None.
        
        Logs:
        - Precision, recall, F1 score, AUC, and log loss.
        - Commit URL, environment, and dataset details.
        """
        try:
            # Log parameters
            if params != None:
                mlflow.log_params(params)
            else:
                mlflow.log_param("params", None)

            # Log metrics
            pred_logits = [1 if pred >= 0.5 else 0 for pred in preds]
            mlflow.log_metric("precision", precision_score(y_true, pred_logits))
            mlflow.log_metric("recall", recall_score(y_true, pred_logits))
            mlflow.log_metric("f1", f1_score(y_true, pred_logits))
            mlflow.log_metric("AUC", roc_auc_score(y_true, preds))
            mlflow.log_metric("logloss", log_loss(y_true, preds))

            # Log script URL with version 
            commit_url = "https://github.com/d-sutariya/instacart_next_basket_prediction/tree/" + prev_commit_hash
            mlflow.log_param("repository url", commit_url)

            # Log environment 
            os.system("conda env export > conda.yaml")
            mlflow.log_artifact("conda.yaml")

            # Log dataset 
            dataset_path = "https://www.kaggle.com/datasets/deepsutariya/instacart-exp-data" 
            mlflow.log_param("dataset url", dataset_path)

            mlflow.end_run()

        except Exception as e:
            raise RuntimeError(f"Error logging model details: {str(e)}")
    
    def train_h2o_glm(self, prev_commit_hash, dataset_version, model_version, params=None):
        """
        Trains a Generalized Linear Model (GLM) using H2O's binomial family.
        
        Parameters:
        prev_commit_hash (str): The commit hash for version control.
        dataset_version (str): The version of the dataset.
        model_version (str): The version of the model.
        params (dict, optional): Hyperparameters for the model. Defaults to None.
        
        Returns:
        h2o_model: The trained H2O GLM model.
        """
        try:
            start = time.time()
            
            h2o_logistic_model = H2OGeneralizedLinearEstimator(family='binomial') \
                                .train(x=self.train_set.drop("reordered").columns, y='reordered', training_frame=self.train_set)
            duration = time.time() - start
            
            with mlflow.start_run():
                mlflow.h2o.log_model(h2o_logistic_model, "h2o_logistic_model")
                mlflow.log_param("family", "binomial")
                mlflow.log_param("alpha", h2o_logistic_model.get_params()['alpha'])
                mlflow.log_param("lambda", h2o_logistic_model.get_params()['lambda'])
                
                mlflow.log_param("training_time", duration)
                mlflow.set_tag("dataset_version", dataset_version)
                mlflow.set_tag("model_version", model_version)
                mlflow.set_tag("algorithm", "h2o_glm")
                
                progress = h2o_logistic_model.scoring_history().to_dict()
                with open("loss_history.json", "w") as f:
                    json.dump(progress, f)
                mlflow.log_artifact("loss_history.json")
                
                if self.test_set != None:
                    preds = h2o_logistic_model.predict(self.test_set).as_data_frame(use_multi_thread=True)['p1']
                    y_true = self.test_set['reordered'].as_data_frame(use_multi_thread=True)
                else:
                    preds = h2o_logistic_model.predict(self.train_set).as_data_frame(use_multi_thread=True)['p1']
                    y_true = self.train_set['reordered'].as_data_frame(use_multi_thread=True)

                self.__log_details(y_true, preds, prev_commit_hash, params)
                
                del y_true, preds
                gc.collect()

            return h2o_logistic_model
        
        except Exception as e:
            raise RuntimeError(f"Error training H2O GLM: {str(e)}")

    def train_h2o_gbm(self, prev_commit_hash, dataset_version, model_version, params=None):
        """
        Trains a Gradient Boosting Machine (GBM) model using H2O's GradientBoostingEstimator.
        
        Parameters:
        prev_commit_hash (str): The commit hash for version control.
        dataset_version (str): The version of the dataset.
        model_version (str): The version of the model.
        params (dict, optional): Hyperparameters for the model. Defaults to None.
        
        Returns:
        h2o_model: The trained H2O GBM model.
        """
        try:
            if params != None and "distribution" not in params.keys():
                params['distribution'] = 'bernoulli'

            if params is None:
                params = {'distribution': 'bernoulli'}
            
            start = time.time()
            
            if self.test_set != None:
                h2o_gbm = H2OGradientBoostingEstimator(**params) \
                          .train(x=self.train_set.drop("reordered").columns,
                                 y='reordered',
                                 training_frame=self.train_set,
                                 validation_frame=self.test_set)
            else:
                h2o_gbm = H2OGradientBoostingEstimator(**params) \
                          .train(x=self.train_set.drop("reordered").columns,
                                 y='reordered',
                                 training_frame=self.train_set)

            duration = time.time() - start
            
            with mlflow.start_run():
                mlflow.h2o.log_model(h2o_gbm, "h2o_gbm_model")
                mlflow.log_params(params if params != None else {})
                mlflow.log_param("training_time", duration)
                mlflow.set_tag("dataset_version", dataset_version)
                mlflow.set_tag("model_version", model_version)
                mlflow.set_tag("algorithm", "h2o_gbm")
                
                progress = h2o_gbm.scoring_history().to_dict()
                with open("loss_history.json", "w") as f:
                    json.dump(progress, f)
                mlflow.log_artifact("loss_history.json")
                
                if self.test_set != None:
                    preds = h2o_gbm.predict(self.test_set).as_data_frame(use_multi_thread=True)['p1']
                    y_true = self.test_set['reordered'].as_data_frame(use_multi_thread=True)
                else:
                    preds = h2o_gbm.predict(self.train_set).as_data_frame(use_multi_thread=True)['p1']
                    y_true = self.train_set['reordered'].as_data_frame(use_multi_thread=True)

                self.__log_details(y_true, preds, prev_commit_hash, params, h2o_gbm)

            del y_true, preds
            gc.collect()
            return h2o_gbm
        
        except Exception as e:
            raise RuntimeError(f"Error training H2O GBM: {str(e)}")

    def train_xgb_gbm(self, prev_commit_hash, dataset_version, model_version, params=None):
        """
        Trains a Gradient Boosting Machine (GBM) using XGBoost.
        
        Parameters:
        prev_commit_hash (str): The commit hash for version control.
        dataset_version (str): The version of the dataset.
        model_version (str): The version of the model.
        params (dict, optional): Hyperparameters for the model. Defaults to None.
        
        Returns:
        xgb_model: The trained XGBoost GBM model.
        """
        try:
            if params != None:
                if 'booster' not in params.keys():
                    params['booster'] = 'gbtree'
                if 'tree_method' not in params.keys():
                    params['tree_method'] = 'hist'
                if 'objective' not in params.keys():
                    params['objective'] = 'binary:logistic'
                if 'eval_metric' not in params.keys():
                    params['eval_metric'] = 'logloss'

            start = time.time()

            if self.test_set != None:
                watchlist = [(self.train_set, 'train'), (self.test_set, 'eval')]
                xgb_model = xgb.train(params, self.train_set, num_boost_round=500, early_stopping_rounds=30, evals=watchlist)
            else:
                xgb_model = xgb.train(params, self.train_set, num_boost_round=500, early_stopping_rounds=30,evals=[(self.train_set,'train')])

            duration = time.time() - start

            with mlflow.start_run():
                
                mlflow.xgboost.log_model(xgb_model, "xgb_gbm_model")
                mlflow.log_params(params if params != None else {})
                mlflow.log_param("training_time", duration)
                mlflow.set_tag("dataset_version", dataset_version)
                mlflow.set_tag("model_version", model_version)
                mlflow.set_tag("algorithm", "xgb_gbm")

                if self.test_set != None:
                    preds = xgb_model.predict(self.test_set)
                    y_true = self.test_set.get_label()
                else:
                    preds = xgb_model.predict(self.train_set)
                    y_true = self.train_set.get_label()

                self.__log_details(y_true, preds, prev_commit_hash, params, xgb_model)

            del y_true, preds
            gc.collect()
            return xgb_model
        
        except Exception as e:
            raise RuntimeError(f"Error training XGBoost GBM: {str(e)}")

    def train_xgb_rf(self, prev_commit_hash, dataset_version, model_version, params=None):
        """
        Trains a Random Forest using XGBoost's 'random forest' booster.
        
        Parameters:
        prev_commit_hash (str): The commit hash for version control.
        dataset_version (str): The version of the dataset.
        model_version (str): The version of the model.
        params (dict, optional): Hyperparameters for the model. Defaults to None.
        
        Returns:
        xgb_model: The trained XGBoost Random Forest model.
        """
        try:
            if params != None:
                if 'booster' not in params.keys():
                    params['booster'] = 'gbtree'
                if 'objective' not in params.keys():
                    params['objective'] = 'binary:logistic'
                if 'eval_metric' not in params.keys():
                    params['eval_metric'] = 'logloss'

            start = time.time()
           
            if self.test_set != None:
                watchlist = [(self.train_set, 'train'), (self.test_set, 'eval')]
                xgb_rf_model = xgb.train(params, self.train_set, num_boost_round=500, early_stopping_rounds=30, evals=watchlist)
            else:
                xgb_rf_model = xgb.train(params, self.train_set, num_boost_round=500, early_stopping_rounds=30,evals=[(self.train_set,'train')])

            duration = time.time() - start

            with mlflow.start_run():
                mlflow.xgboost.log_model(xgb_rf_model, "xgb_rf_model")
                mlflow.log_params(params if params != None else {})
                mlflow.log_param("training_time", duration)
                mlflow.set_tag("dataset_version", dataset_version)
                mlflow.set_tag("model_version", model_version)
                mlflow.set_tag("algorithm", "xgb_rf")

                if self.test_set != None:
                    preds = xgb_rf_model.predict(self.test_set)
                    y_true = self.test_set.get_label()
                else:
                    preds = xgb_rf_model.predict(self.train_set)
                    y_true = self.test_set.get_label()

                self.__log_details(y_true, preds, prev_commit_hash, params, xgb_rf_model)

            del y_true, preds
            gc.collect()
            return xgb_rf_model
        
        except Exception as e:
            raise RuntimeError(f"Error training XGBoost RF: {str(e)}")

    def train_lgbm(self, prev_commit_hash, dataset_version, model_version, params=None):
        """
        Trains a LightGBM model.
        
        Parameters:
        prev_commit_hash (str): The commit hash for version control.
        dataset_version (str): The version of the dataset.
        model_version (str): The version of the model.
        params (dict, optional): Hyperparameters for the model. Defaults to None.
        
        Returns:
        lgb_model: The trained LightGBM model.
        """
        try:
            if params != None:
                if 'objective' not in params.keys():
                    params['objective'] = 'binary'

            start = time.time()

            
            if self.test_set != None:
                lgb_model = lgb.train(params, self.train_set, num_boost_round=500, early_stopping_rounds=30, valid_sets=[self.test_set])
            else:
                lgb_model = lgb.train(params, self.train_set, num_boost_round=500, early_stopping_rounds=30, valid_sets=[self.train_set])

            duration = time.time() - start

            with mlflow.start_run():
                
                mlflow.lightgbm.log_model(lgb_model, "lgb_model")
                mlflow.log_params(params if params != None else {})
                mlflow.log_param("training_time", duration)
                mlflow.set_tag("dataset_version", dataset_version)
                mlflow.set_tag("model_version", model_version)
                mlflow.set_tag("algorithm", "lgbm")

            if self.test_set != None:
                
                preds = lgb_model.predict(self.test_set.get_data())
                y_true = self.test_set.get_label()
    
                self.__log_details(y_true, preds, prev_commit_hash, params, lgb_gbm)
            else:
                
                preds = lgb_model.predict(self.train_set.get_data())
                y_true = self.train_set.get_label()
    
                self.__log_details(y_true, preds, prev_commit_hash, params, lgb_gbm)

            del y_true, preds
            gc.collect()
            return lgb_model
        
        except Exception as e:
            raise RuntimeError(f"Error training LightGBM: {str(e)}")


In [14]:
if train_xgb_gbm == True:
    model_trainer = ModelTrainer("final_instacart_training",dtrain_2,params)
    # paramss = {
    #     'max_depth':1,
    #     'min_child_weight':5,
    #     'gamma':0.01,
    #     'lambda':100,
    #     'alpha':100
    # }
    xgb_gbm = model_trainer.train_xgb_gbm("c391e8337f10ceb5870cb639159539f5e3497fbf",dataset_version,model_version,params)

if train_xgb_rf == True:
    model_trainer = ModelTrainer("instacart_training",dtrain_2,dtest_2,"reordered")
    xgb_rf = model_trainer.train_xgb_rf("c391e8337f10ceb5870cb639159539f5e3497fbf",dataset_version,model_version)

    

RuntimeError: Error training XGBoost GBM: ('invalid cache item: dict', [<xgboost.core.DMatrix object at 0x7ca942fe4400>, <xgboost.core.DMatrix object at 0x7ca942fe4400>, {'max_depth': 1, 'min_child_weight': 389, 'verbose': -1, 'gamma': 438, 'eta': 0.0907183045377987, 'subsample': 0.10741019033611729, 'colsample_bytree': 0.336917979846298, 'colsample_bylevel': 0.118177830385349, 'lambda': 11, 'alpha': 89, 'booster': 'gbtree', 'tree_method': 'hist', 'objective': 'binary:logistic', 'eval_metric': 'logloss'}])

In [ ]:
read_as_lgb_dataset = False
train_lgb = False

In [ ]:
if read_as_lgb_dataset == True:

    # train_features_name.remove('reordered')
#     test_features_name.remove('reordered')
    
    dtrain_2 = lgb.Dataset("/kaggle/input/final-dataset-generator/final_prior_train_set.csv/part-00000-95d78719-42e1-441c-a4e1-d662676226a3-c000.csv",params={'label_column':train_label_index},free_raw_data=False).construct()
    dtest_2 = pd.read_csv("/kaggle/input/final-dataset-generator/featured_test_set.csv/part-00000-1b691469-02d1-4ed1-8b50-ec568a4bf8a9-c000.csv",header = None)


In [ ]:
if train_lgb == True:
    model_trainer = ModelTrainer("final_instacart_training",[dtrain_2])
    lgb_gbm = model_trainer.train_lgb_gbm("c391e8337f10ceb5870cb639159539f5e3497fbf",dataset_version,model_version)

In [ ]:
read_as_h2o_frame = False
train_h2o = False

In [ ]:
if read_as_h2o_frame == True:
    h2o.init(max_mem_size = '25G')
    dtrain_2 = h2o.import_file("/kaggle/input/instacart-dataset-transformer-cv2/final_prior_train_set.csv/part-00000-444aa52f-9c41-4a07-8342-7f82d9fcf6ac-c000.csv")
    dtest_2 = h2o.import_file("/kaggle/input/instacart-dataset-transformer-cv2/featured_test_set.csv/part-00000-017a6365-a37c-4ce4-8bf4-96e8daa33570-c000.csv")
    dtrain_2.col_names = train_features_name
    dtest_2.col_names = test_features_name
    dtrain_2['reordered'] = dtrain_2['reordered'].asfactor()
    dtest_2['reordered'] = dtest_2['reordered'].asfactor()

In [ ]:
if train_h2o == True:
    
    model_trainer = ModelTrainer("instacart_training",[dtrain_2],dtest_2,"reordered")
    h2o_gbm = model_trainer.train_h2o_glm("c391e8337f10ceb5870cb639159539f5e3497fbf",dataset_version,model_version)